In [15]:
from main import get_miner
from log_parsing import from_id_to_template
import joblib
from data_integrate import *
from pattern_miner import *
from pattern_ranker import *
import pandas as pd

In [2]:
log_template_miner = get_miner()

In [3]:
res_dict = joblib.load('output/fault_free_eventgraphs.joblib')

In [4]:
res_dict2 = joblib.load('output/fault_suffering_eventgraphs.joblib')

In [5]:
res_dict2.keys()

dict_keys(['2022-08-22 03:54', '2022-08-22 03:55', '2022-08-22 04:03', '2022-08-22 04:04', '2022-08-22 04:11', '2022-08-22 04:12', '2022-08-22 04:19', '2022-08-22 04:20', '2022-08-22 04:28', '2022-08-22 04:29', '2022-08-22 04:37', '2022-08-22 04:38', '2022-08-22 04:45', '2022-08-22 04:46', '2022-08-22 04:53', '2022-08-22 04:54', '2022-08-22 05:02', '2022-08-22 05:03', '2022-08-22 05:10', '2022-08-22 05:11', '2022-08-22 05:19', '2022-08-22 05:20', '2022-08-22 05:28', '2022-08-22 05:29', '2022-08-22 05:36', '2022-08-22 05:37', '2022-08-22 05:44', '2022-08-22 05:45', '2022-08-22 05:53', '2022-08-22 05:54', '2022-08-22 06:02', '2022-08-22 06:03', '2022-08-22 06:36', '2022-08-22 06:37', '2022-08-22 06:53', '2022-08-22 06:54', '2022-08-22 07:02', '2022-08-22 07:03', '2022-08-22 07:11', '2022-08-22 07:12', '2022-08-22 07:18', '2022-08-22 07:19', '2022-08-22 07:27', '2022-08-22 07:28', '2022-08-22 07:45', '2022-08-22 07:46', '2022-08-22 07:54', '2022-08-22 07:55', '2022-08-23 12:02', '2022-08-

In [6]:
abnormal_dates = res_dict2.keys()

In [23]:
abnormal_dates = res_dict2.keys()
features_labeled = []
for normal_date in res_dict.keys():
    date = normal_date.split(' ')[0]
    fault_free_pattern = get_pattern_support(res_dict[normal_date])
    print(date)
    for abnormal_date in [d for d in abnormal_dates if d.split(' ')[0] == date]:
        print(abnormal_date)
        fault_suffering_event_graphs = res_dict2[abnormal_date]['event_graphs']
        fault_suffering_pattern = get_pattern_support(fault_suffering_event_graphs)
        total_score = sum(list(fault_suffering_pattern.values()))
        ranking = pattern_ranker_custom(fault_free_pattern, fault_suffering_event_graphs, fault_suffering_pattern, log_template_miner)
        res = { 
               'abnormal_time': abnormal_date,
               'abnormal_patternnoerror_ranked': ranking[0],
               'abnormal_patternerror_ranked': ranking[1], 
               'total_score': total_score,
               'inject_time': res_dict2[abnormal_date]['inject_time'],
               'inject_pod': res_dict2[abnormal_date]['inject_pod'], 
               'inject_type': res_dict2[abnormal_date]['inject_type'],
            }
        features_labeled.append(res)

2022-08-22
2022-08-22 03:54
2022-08-22 03:55
2022-08-22 04:03
2022-08-22 04:04
2022-08-22 04:11
2022-08-22 04:12
2022-08-22 04:19
2022-08-22 04:20
2022-08-22 04:28
2022-08-22 04:29
2022-08-22 04:37
2022-08-22 04:38
2022-08-22 04:45
2022-08-22 04:46
2022-08-22 04:53
2022-08-22 04:54
2022-08-22 05:02
2022-08-22 05:03
2022-08-22 05:10
2022-08-22 05:11
2022-08-22 05:19
2022-08-22 05:20
2022-08-22 05:28
2022-08-22 05:29
2022-08-22 05:36
2022-08-22 05:37
2022-08-22 05:44
2022-08-22 05:45
2022-08-22 05:53
2022-08-22 05:54
2022-08-22 06:02
2022-08-22 06:03
2022-08-22 06:36
2022-08-22 06:37
2022-08-22 06:53
2022-08-22 06:54
2022-08-22 07:02
2022-08-22 07:03
2022-08-22 07:11
2022-08-22 07:12
2022-08-22 07:18
2022-08-22 07:19
2022-08-22 07:27
2022-08-22 07:28
2022-08-22 07:45
2022-08-22 07:46
2022-08-22 07:54
2022-08-22 07:55
2022-08-23
2022-08-23 12:02
2022-08-23 12:03
2022-08-23 12:11
2022-08-23 12:12
2022-08-23 12:19
2022-08-23 12:20
2022-08-23 12:28
2022-08-23 12:29
2022-08-23 12:36
2022-08-2

In [24]:
# Convert to DataFrame
df = pd.DataFrame(features_labeled)
# locate templates with err or fail in string, treat them special (dont lose on ranking)

In [25]:
df.head()

,abnormal_time,abnormal_patternnoerror_ranked,abnormal_patternerror_ranked,total_score,inject_time,inject_pod,inject_type
0,2022-08-22 03:54,"[{'events': '5_12', 'score': 2213, 'deepth': 1...",[],59065,2022-08-22 03:53:54,frontend-579b9bff58-t2dbm,cpu_contention
1,2022-08-22 03:55,"[{'events': '5_12', 'score': 1724, 'deepth': 1...",[],43804,2022-08-22 03:53:54,frontend-579b9bff58-t2dbm,cpu_contention
2,2022-08-22 04:03,"[{'events': '47_48', 'score': 1676, 'deepth': ...","[{'events': '2_185', 'score': 261, 'deepth': 1...",29943,2022-08-22 04:02:07,frontend-579b9bff58-t2dbm,return
3,2022-08-22 04:04,"[{'events': '47_48', 'score': 1471, 'deepth': ...","[{'events': '2_185', 'score': 255, 'deepth': 1...",34042,2022-08-22 04:02:07,frontend-579b9bff58-t2dbm,return
4,2022-08-22 04:11,"[{'events': '5_12', 'score': 2010, 'deepth': 1...",[],50771,2022-08-22 04:10:20,frontend-579b9bff58-t2dbm,cpu_consumed


In [26]:
df.to_csv('output/features_labeled_v1.csv', index=False)